### RQ1 - Comparing different quantization levels

**Quantization Dimension**

How does quantization in different models and architectures affect system and task-specific metrics.

In [3]:
from benchmark.benchmark import ModelBenchmark
import torch


def run_benchmark(backend, model_name, task, base_path, samples=500, verbose=False, batch_size=100):
    print(f"Running benchmark for {model_name} with {backend} on {task}")
    try:
        bm = ModelBenchmark(
            backend=backend,
            model_name=model_name,
            model_path=f"{base_path}/models/{model_name}",
            base_path=base_path,
            task=task,
            verbose=verbose,
        )
        bm.run(samples=samples, batch_size=batch_size)
        bm.close()
        del bm
        torch.cuda.empty_cache()
        print(f"✅ Completed: {model_name} | {backend} | {task}")
    except Exception as e:
        print(f"❌ Failed: {model_name} | {backend} | {task} -- {e}")
        torch.cuda.empty_cache()  # ensure no memory leak on error

2025-05-12 15:48:54.090065: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747064934.110013  260255 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747064934.115811  260255 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747064934.135655  260255 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747064934.135683  260255 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747064934.135685  260255 computation_placer.cc:177] computation placer alr

INFO 05-12 15:48:57 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-12 15:48:57 [__init__.py:239] Automatically detected platform cuda.
[2025-05-12 15:49:00,664] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [7]:
base_path = "/home/ubuntu/fast_llm_inference/"

backends = ["vllm"] #, "huggingface","deepspeed_mii", "llama.cpp"]
models   = [
    "llama-3.1-8B-Instruct",
    "llama-3.1-8B-Instruct-4bit",
    "llama-3.1-8B-Instruct-8bit",
    "llama-3.2-3b-instruct",
    "llama-3.2-1b-instruct",
    "llama-3.2-3b-instruct-4bit",
    "llama-3.2-1b-instruct-4bit",
    "llama-3.2-3b-instruct-8bit",
    "llama-3.2-1b-instruct-8bit",
   
    "Qwen2.5-7B-Instruct",
    "Qwen2.5-7B-Instruct-4bit",
    "Qwen2.5-7B-Instruct-8bit", # some weird error
    "Qwen2.5-3B-Instruct",
    "Qwen2.5-1.5B-Instruct",
    "Qwen2.5-0.5B-Instruct",
    "Qwen2.5-3B-Instruct-4bit",
    "Qwen2.5-1.5B-Instruct-4bit",
    "Qwen2.5-0.5B-Instruct-4bit",
    "Qwen2.5-3B-Instruct-8bit",
    "Qwen2.5-1.5B-Instruct-8bit",
    "Qwen2.5-0.5B-Instruct-8bit",


    "gemma-2-9b-it-bnb4",
    "gemma-2-9b-it-8bit",
    "gemma-2-9b-it", # too large
    "gemma-2-2b-it-4bit",
    "gemma-2-2b-it-8bit",
    "gemma-2-2b-it",
]

tasks    = ["summarization", "qa", "sql",]

first run

In [ ]:
for backend in backends:
    for model in models:
        for task in tasks:
            run_benchmark(
                backend=backend,
                model_name=model,
                task=task,
                base_path=base_path,
                samples=100,
                verbose=False,
                batch_size=100,
            )

check if anything is missing

In [12]:
import os

# Define your parameters
backends = ["vllm"]
models = [
    "llama-3.1-8B-Instruct",
    "llama-3.1-8B-Instruct-4bit",
    "llama-3.1-8B-Instruct-8bit",
    "llama-3.2-3b-instruct",
    "llama-3.2-1b-instruct",
    "llama-3.2-3b-instruct-4bit",
    "llama-3.2-1b-instruct-4bit",
    "llama-3.2-3b-instruct-8bit",
    "llama-3.2-1b-instruct-8bit",
    "Qwen2.5-7B-Instruct",
    "Qwen2.5-7B-Instruct-4bit",
    "Qwen2.5-7B-Instruct-8bit",
    "Qwen2.5-3B-Instruct",
    "Qwen2.5-1.5B-Instruct",
    "Qwen2.5-0.5B-Instruct",
    "Qwen2.5-3B-Instruct-4bit",
    "Qwen2.5-1.5B-Instruct-4bit",
    "Qwen2.5-0.5B-Instruct-4bit",
    "Qwen2.5-3B-Instruct-8bit",
    "Qwen2.5-1.5B-Instruct-8bit",
    "Qwen2.5-0.5B-Instruct-8bit",
    "gemma-2-9b-it-4bit",
    "gemma-2-9b-it-8bit",
    "gemma-2-9b-it",
    "gemma-2-2b-it-4bit",
    "gemma-2-2b-it-8bit",
    "gemma-2-2b-it",
]
tasks = ["summarization", "qa", "sql"]

results_dir = "./results/experiment_1/"

missing_models = set()

for backend in backends:
    for model in models:
        for task in tasks:
            filename = f"{backend}_{model}_{task}.csv"
            filepath = os.path.join(results_dir, filename)
            if not os.path.exists(filepath):
                missing_models.add(model)
                break  # No need to check more tasks if one is missing

# Print models with missing files
if missing_models:
    print("Models with missing files:")
    for model in sorted(missing_models):
        print(model)
else:
    print("✅ All models are complete.")

Models with missing files:
Qwen2.5-7B-Instruct-8bit
gemma-2-9b-it


try again with the missing models

In [ ]:
for backend in backends:
    for model in list(missing_models):
        for task in tasks:
            run_benchmark(
                backend=backend,
                model_name=model,
                task=task,
                base_path=base_path,
                samples=500,
                verbose=False,
                batch_size=100,
            )

### RQ2 - Comparing different inference engines

**Framework Dimension** 

Which inference framework (Transformers, vLLM, DeepSpeed MII,172
LMDeploy, llama.cpp) strikes the best balance between system resource usage (e.g., GPU173
utilization, joules/token) and system performance (tokens/s)?174

In [ ]:
base_path = "/home/ubuntu/fast_llm_inference/models"

backends = ["vllm", "huggingface", "llama.cpp"] #,"deepspeed_mii", "huggingface"]

models   = [
    "gemma-2-9b-it", 
    "gemma-2-2b-it",

    "llama-3.1-8B-Instruct",
    "llama-3.2-3b-instruct",
    "llama-3.2-1b-instruct",
   
    "Qwen2.5-7B-Instruct",
    "Qwen2.5-3B-Instruct",
    "Qwen2.5-1.5B-Instruct",
    "Qwen2.5-0.5B-Instruct",
]

tasks    = ["summarization", "qa", "sql",]

for backend in backends:
    if backend == "llama.cpp":
        models = [
            "gemma-2-2b-it-fp16.gguf",
            "gemma-2-9b-it-fp16.gguf",

            "llama-3.1-8B-Instruct-f16.gguf",
            "Llama-3.2-1B-Instruct-f16.gguf",
            "Llama-3.2-3B-Instruct-f16.gguf",
            
            "qwen2.5-0.5b-instruct-fp16.gguf",
            "qwen2.5-1.5b-instruct-fp16.gguf",
            "qwen2.5-3b-instruct-fp16.gguf",
            "qwen2.5-7B-instruct-fp16.gguf",
        ]
        
    for model in models:
        for task in tasks:
            run_benchmark(
                backend=backend,
                model_name=model,
                task=task,
                base_path=base_path,
                samples=100,
                verbose=False,
                batch_size=20,
            )

#### RQ 3 - Comparing different use cases

**Scenario/Workload Dimension**

How do locally deployed LLMs and inference backends
perform and scale across the three dominant inference scenarios—single - stream (single user),
batched offline processing, and multi- user server workloads? Do system metrics – throughput,
GPU utilization, joules/token — evolve as the average number of queries per second varies
over time?

In [ ]:
import torch
from benchmark.benchmark import ModelBenchmark

def run_benchmark(
    backend: str,
    model_name: str,
    task: str,
    base_path: str,
    scenario: str = "batch",            # "single", "batch", or "server"
    run_time: float = None,             # only for server: total time in seconds
    requests_per_sec: float = None,     # only for server: λ (req/s)
    batch_size: int = 100,              # only for batch
    max_batch_size: int = None,         # only for server: cap per-batch size
    sample_interval: float = 0.1,       # telemetry interval (s)
    verbose: bool = False
):
    print(f"Running benchmark for {model_name} with {backend} on {task} [{scenario}]")
    try:
        bm = ModelBenchmark(
            backend=backend,
            model_name=model_name,
            model_path=f"{base_path}/models/{model_name}",
            base_path=base_path,
            task=task,
            verbose=verbose,
        )

        if scenario == "server":
            assert run_time is not None,    "Must set run_time in server mode"
            assert requests_per_sec is not None, "Must set requests_per_sec in server mode"
            df = bm.run(
                scenario="server",
                run_time=run_time,
                requests_per_sec=requests_per_sec,
                sample_interval=sample_interval,
                max_batch_size=max_batch_size
            )

        elif scenario == "single":
            df = bm.run(
                samples=None,        # samples ignored in single
                batch_size=1,
                scenario="single",
                sample_interval=sample_interval
            )

        elif scenario == "batch":
            df = bm.run(
                samples=None,        # samples ignored in batch
                batch_size=batch_size,
                scenario="batch",
                sample_interval=sample_interval
            )
        else:
            raise ValueError(f"Unknown scenario: {scenario}")

        # optional: inspect or save df here
        bm.close()
        del bm
        torch.cuda.empty_cache()
        print(f"✅ Completed: {model_name} | {backend} | {task} | {scenario}")
        return df

    except Exception as e:
        print(f"❌ Failed: {model_name} | {backend} | {task} | {scenario} -- {e}")
        torch.cuda.empty_cache()
        return None


2025-05-14 11:41:39.338320: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747222899.358381  623744 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747222899.363479  623744 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747222899.378544  623744 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747222899.378563  623744 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747222899.378565  623744 computation_placer.cc:177] computation placer alr

INFO 05-14 11:41:42 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-14 11:41:42 [__init__.py:239] Automatically detected platform cuda.
[2025-05-14 11:41:45,774] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [ ]:
base_path = "/home/ubuntu/fast_llm_inference"

# simulate a 100-request workload at 5 QPS → run_time ≈ 100/5 = 20 s
run_benchmark(
    backend="vllm",
    model_name="gemma-2-2b-it",
    task="qa",
    base_path=base_path,
    scenario="server",
    run_time=60.0,            # total simulation time in seconds
    requests_per_sec=5,     # λ = 5 requests/sec
    sample_interval=0.05,     # polling every 50 ms
    max_batch_size=64        # cap each batch to 32 prompts
)


Running benchmark for gemma-2-2b-it with vllm on qa [server]


INFO 05-14 11:33:59 [config.py:717] This model supports multiple tasks: {'reward', 'classify', 'generate', 'score', 'embed'}. Defaulting to 'generate'.
INFO 05-14 11:33:59 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 05-14 11:34:00 [utils.py:2382] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/getting_started/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 05-14 11:34:05 [__init__.py:239] Automatically detected platform cuda.


2025-05-14 11:34:06.308466: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747222446.325800  622886 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747222446.330805  622886 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747222446.345199  622886 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747222446.345214  622886 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747222446.345216  622886 computation_placer.cc:177] computation placer alr

INFO 05-14 11:34:13 [core.py:58] Initializing a V1 LLM engine (v0.8.5.post1) with config: model='/home/ubuntu/fast_llm_inference/models/gemma-2-2b-it', speculative_config=None, tokenizer='/home/ubuntu/fast_llm_inference/models/gemma-2-2b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=None, served_model_name=/home/ubuntu/fast_llm_inference/models/gemma-2-2b-it, num_scheduler_steps=1, multi

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.57s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.57s/it]



...

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Stats for gemma-2-2b-it on vllm/qa:
prompt_length                       1393.740000 ± 302.397244
TTFT                                     0.028277 ± 0.000805
ATL                                      0.105542 ± 0.047617
GL                                       0.188673 ± 0.052383
TPS                                     11.364900 ± 4.646620
SPS                                      6.105700 ± 1.540733
Avg GPU Mem (MB)                     21353.873300 ± 0.067000
Peak GPU Mem (MB)                    21353.880000 ± 0.000000
Avg GPU Util (%)                       80.804100 ± 13.714770
Peak GPU Util (%)                      86.070000 ± 11.817001
Total Energy (Wh)                        0.003468 ± 0.001053
Avg Power (W)                           66.182000 ± 6.972507
Peak Power (W)                          67.279800 ± 6.415855
Energy per Token (J/token)               6.998406 ± 3.297333
Energy per Sentence (J/sentence)        11.598829 ± 3.657829
Memory Usage (MB)                    21353.880000

[rank0]:[W514 11:36:09.940427095 ProcessGroupNCCL.cpp:1496] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


✅ Completed: gemma-2-2b-it | vllm | qa | server


,prompt_length,prompt,generated_answer,reference_answer,TTFT,ATL,GL,TPS,SPS,Avg GPU Mem (MB),...,Total Energy (Wh),Avg Power (W),Peak Power (W),Energy per Token (J/token),Energy per Sentence (J/sentence),Memory Usage (MB),Model Size (MB),Overhead (MB),exact_match,F1_score
0,1275,### SYSTEM\nYou are a question-answering assis...,Lothar de Maizière,"[Lothar de Maizière, Lothar de Maizière, Lotha...",0.0303,0.0980,0.2941,10.20,3.40,21353.21,...,0.002317,28.37,30.88,2.780889,8.342668,21353.88,5007.298138,16346.581862,1,1.000000
1,753,### SYSTEM\nYou are a question-answering assis...,Complexity classes,"[complexity classes, complexity classes, some ...",0.0280,0.0736,0.1472,13.58,6.79,21353.88,...,0.001388,33.95,35.16,2.499108,4.998216,21353.88,5007.298138,16346.581862,1,1.000000
2,1197,### SYSTEM\nYou are a question-answering assis...,GTE,[Telenet was incorporated in 1973 and started ...,0.0280,0.1589,0.1589,6.29,6.29,21353.88,...,0.001772,40.16,51.23,6.379486,6.379486,21353.88,5007.298138,16346.581862,1,1.000000
3,1541,### SYSTEM\nYou are a question-answering assis...,Water flow,"[water flow through the body cavity, κτείς kte...",0.0283,0.0787,0.1575,12.70,6.35,21353.88,...,0.002300,52.56,52.69,4.139407,8.278814,21353.88,5007.298138,16346.581862,0,0.571429
4,1747,### SYSTEM\nYou are a question-answering assis...,1705,"[12 May 1705, 1705, 12 May 1705]",0.0321,0.2027,0.2027,4.93,4.93,21353.88,...,0.003175,56.39,67.79,11.430910,11.430910,21353.88,5007.298138,16346.581862,1,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1807,### SYSTEM\nYou are a question-answering assis...,Ctenophores,"[ctenophores,, ctenophores, ctenophores]",0.0281,0.1855,0.1855,5.39,5.39,21353.88,...,0.003433,66.63,66.79,12.357441,12.357441,21353.88,5007.298138,16346.581862,1,1.000000
96,1065,### SYSTEM\nYou are a question-answering assis...,Great Dividing Range,"[Great Dividing Range, the Great Dividing Rang...",0.0281,0.0659,0.1977,15.17,5.06,21353.88,...,0.003866,70.39,71.03,4.638959,13.916877,21353.88,5007.298138,16346.581862,1,1.000000
97,1788,### SYSTEM\nYou are a question-answering assis...,Central Asian Muslims,"[Central Asian Muslims, Central Asian Muslims,...",0.0286,0.0612,0.1836,16.34,5.45,21353.88,...,0.003632,71.23,71.53,4.358907,13.076720,21353.88,5007.298138,16346.581862,1,1.000000
98,1122,### SYSTEM\nYou are a question-answering assis...,August 2004,"[August 2004, August 2004, August 2004]",0.0281,0.1238,0.2477,8.07,4.04,21353.88,...,0.004814,69.96,71.12,8.664640,17.329280,21353.88,5007.298138,16346.581862,1,1.000000
